<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/get_resource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Accessing a resource

Accessing a REST resource through Python is very easy when using the Requests
library. It's a very flexible library that will make our lives a lot easier.

The basic idea consists of making a request and reading data from the response. When accessing the Onroerend Erfgoed webservices, the data will almost always be transferred as JSON.

In [9]:
import requests
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print the response's JSON data
print(json.dumps(data, sort_keys=True, indent=4))

{
    "alternatieve_namen": [],
    "bijlagen": [],
    "current_state": 75,
    "disciplines": [
        {
            "id": 2,
            "naam": "Bouwkundig"
        }
    ],
    "erfgoedwaarde": true,
    "externe_linken": [
        {
            "id": 2,
            "in_te_vullen_id": "335263769",
            "omschrijving": "Red Star Line 1, Antwerpen | Het Red Star Line 1-gebouw in A\u2026 | Flickr",
            "url": "https://www.flickr.com/photos/erfgoed/335263769"
        },
        {
            "id": 2,
            "in_te_vullen_id": "344676052",
            "omschrijving": "Red Star Line 2, Antwerpen | Het Red Star Line 2-gebouw in A\u2026 | Flickr",
            "url": "https://www.flickr.com/photos/erfgoed/344676052"
        },
        {
            "id": 2,
            "in_te_vullen_id": "344685372",
            "omschrijving": "Red Star Line 3, Antwerpen | Het Red Star Line 3-gebouw in A\u2026 | Flickr",
            "url": "https://www.flickr.com/photos/erfgoed/344685

This example ask for erfgoedobject 200498, the [Red Star Line buildings](https://inventaris.onroerenderfgoed.be/erfgoedobjecten/200498). It returns all data available on this object.

The Inventaris makes a distinction between three main types of objects:
* **Erfgoedobjecten** (heritage objects): Parts of our heritage, seen in a way that makes sense to humans. By definition somewhat vague.
* **Aanduidingsobjecten** (designation objects): Pars of our heritage, but seen in a legal way. Heritage as it is known to law. By defnition very precise.
* **Waarnemingen** (observations): Observations concering heritage. Things that were seen, but they might not necessary (still) have heritage value. Mainly used for archaological information.

More information about the [distinction between Erfgoedobject and Aanduidingsobject](https://onroerenderfgoed.github.io/posts/2019/inventaris-erfgoedobjecten-aanduidingsobjecten/). We will focus on the first two, Erfgoedobjecten and Aanduidingsobjecten, and the relations between them. These relations indicate what the legal status is of a certain Erfgoedobject. To detect these, we look at the **relations** attribute of the Erfgoedobject. There are several types of relations, but we are only interested in the ones with Aanduidingsobjecten. We can detect these by looking at the attribute **verwant** of every relation. We only want the ones with **id** 5.

Because scrolling through the whole object all the time is rather unpractical, we only want to print some basic information about an object.




In [ ]:
import requests
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print URI and name of the Erfgoedobject
print(f"{data['uri']}: {data['naam']}")
# Print a short description of an object
print(f"{data['korte_beschrijving']}")

print(f"Designations:")
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    print(f"{r['relatietype']['naam']} {r['aanduidingsobjecttype']} {r['uri']}: {r['omschrijving']}")

In our previous example, we still stuck to the object we had requested before. But there comes a point when we need some information not present in the object we've requested. Then we need to request a new object.

Suppose we're interested in the Deecrees (Besluiten) that led to the designation of a certain object. These are linked to the Aanduidingsobjecten, but in the previous examples we couldn't see them. When we request the Erfgoedobject, we get a summary of every coupled Aanduidingsobject, but not the full object. The full object does contain a link to the Besluit. So, we'll try to fetch those.

In [14]:
import requests
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print URI and name of the Erfgoedobject
print(f"{data['uri']}: {data['naam']}")
# Print a short description of an object
print(f"{data['korte_beschrijving']}")

print(f"Designations:")
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    print(f"* {r['relatietype']['naam']} {r['aanduidingsobjecttype']} {r['uri']}: {r['omschrijving']}")

    response = requests.get(
      r['uri'],
      headers = {'Accept': 'application/json'}
    )

    aanduidingsobjectdata = response.json()

    for besluiten in aanduidingsobjectdata['besluiten']:
      print(f"\t {besluiten['datum_ondertekening']} {besluiten['uri']}: {besluiten['onderwerp']}")

https://id.erfgoed.net/erfgoedobjecten/200498: Rederijgebouwen Red Star Line
De Red Star Line-gebouwen maken deel uit van het magazijnencomplex opgericht door de legendarische scheepvaartmaatschappij Red Star Line. Deze maatschappij vervoerde tussen het einde van de 19de en het begin van de 20ste eeuw bijna drie miljoen passagiers naar de Verenigde Staten en Canada. De gebouwen aan de Rijnkaai fungeerden als één van de belangrijkste vertrekpunten van deze grote emigratiebeweging. Het Red Star Line 1-gebouw werd opgericht in 1893 ter vervanging van een ouder gebouw uit 1873. Het Red Star Line 2-gebouw werd opgericht in 1902, het Red Star Line 3-gebouw in 1921-1922. Na een restauratie opende in 2013 het nieuwe Red Star Line Museum in het complex.
Designations:
* Is aangeduid als Vastgesteld bouwkundig erfgoed https://id.erfgoed.net/aanduidingsobjecten/102630: Rederijgebouwen Red Star Line
	 2019-03-14 https://id.erfgoed.net/besluiten/14618: Besluit tot vaststelling van de inventaris bouw